In [1]:
%matplotlib tk

# Directory manipulation for relative notebook imports
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)

# Relative import for relevant methods
from Visualization.plot_util import *
from Visualization.projection_plots import *
from Data.data_load_util import *
from Simulation.projections_util import *
from Models.Neat.neat_util import *
from Models.MILP.milp_util import *
from Simulation.projections import *


import pickle

In [2]:
# Loading the datasets 
zips_df, state_df, pos_df = make_dataset(granularity='both', remove_outliers=False, load_dir_prefix='../Data/')

#update zips_df with 2025 panels
projection2025 = get_baseline_2025(zips_df, state_df, save="../Data/Baseline_2025/baseline.pkl", load="../Data/Baseline_2025/baseline.pkl")
zips_df = updated_df_with_picks(zips_df, projection2025.panel_placements)

data_manager = DataManager(zips_df, fields=['Median_income', 'carbon_offset_kg_per_panel', 'yearly_sunlight_kwh_kw_threshold_avg', 'panel_utilization', 'black_prop'])


Loading from previous calculations...


In [3]:

milp_data_manager = DataManager(zips_df, fields=['carbon_offset_metric_tons_per_panel', 'yearly_sunlight_kwh_kw_threshold_avg'])

milp_grid = milp_gridsearch(milp_data_manager, MilpModel(), n_panels=2000000, 
                                      max_weights=[5,5,5,5], n_samples=3,
                                      objectives=create_paper_objectives(),
                                      save="./milp_pareto_test_bigger.csv", load = "./milp_pareto_test_bigger.csv"
                                      )


In [4]:
# #test the milf vs regular model
# with open(dir1 + "\\Models\\Neat\\models\\NEAT_model_lexicase_base_weights.pkl", 'rb') as f:
#     network = pickle.load(f)

# neat_proj = create_neat_proj(data_manager, 100000, NeatModel(network), create_paper_objectives())


In [5]:

# with open("test_milp_placement.pkl", 'rb') as f:
#     milp_placement = pickle.load(f)
# # print (milp_placement)
# milp_proj = create_projection_from_panel_assignment(zips_df, milp_placement, create_paper_objectives())

In [6]:

# num_panels = 10000000
#get the NEAT projection
with open(dir1 + "\\Models\\Neat\\models\\05-30-25\\NEAT_model_lexicase_weighted_3.pkl", 'rb') as f:
    network = pickle.load(f)

#2 million panels
neat_proj_path_2m = "../Simulation/Projection_Saves/projections_neat_2m_from_2025.pkl"
neat_proj_new_2m = create_neat_proj(data_manager, 2000000, NeatModel(network), create_paper_objectives(), save=neat_proj_path_2m, load=neat_proj_path_2m)


#10 million panels
# neat_proj_path_10m = "../Simulation/Projection_Saves/projections_neat_10m.pkl"
# neat_proj_new_10m = create_neat_proj(data_manager, 10000000, NeatModel(network), create_paper_objectives(), save=neat_proj_path_10m)#, load=neat_proj_path_10m)

#original model
# with open(dir1+"/Models/Neat/models/01-09-25/NEAT_model2M_lexicase.pkl", 'rb') as f:
#     network = pickle.load(f)


# proj_path2 = "../Simulation/Projection_Saves/projections_neat_1m.pkl"
# neat_proj_original = create_neat_proj(data_manager, num_panels, NeatModel(network), create_paper_objectives(), save=proj_path2, load=proj_path2)



Loading from previous calculations...


In [7]:
#Get all other projections
proj_path_2m = "../Simulation/Projection_Saves/projections_2m_from_2025.pkl"
projections_2m = create_projections(zips_df, state_df, n_panels=2000000, save=proj_path_2m, load=proj_path_2m)
projections_2m[0].objective_projections
# proj_path_10m = "../Simulation/Projection_Saves/projections_10m.pkl"
# projections_10m = create_projections(zips_df, state_df, n_panels=10000000, save=proj_path_10m, load=proj_path_10m)

Loading from previous simulation...


{'Carbon Offset': {0: 0.0,
  1: 0.15061081921849642,
  2: 0.30122163843699284,
  3: 0.4518324576554893,
  4: 0.6024432768739857,
  5: 0.7530540960924821,
  6: 0.9036649153109786,
  7: 1.0542757345294749,
  8: 1.2048865537479714,
  9: 1.3554973729664679,
  10: 1.5061081921849642,
  11: 1.6567190114034607,
  12: 1.8073298306219572,
  13: 1.9579406498404535,
  14: 2.1085514690589497,
  15: 2.2591622882774463,
  16: 2.4097731074959428,
  17: 2.5603839267144393,
  18: 2.7109947459329358,
  19: 2.861605565151432,
  20: 3.0122163843699283,
  21: 3.162827203588425,
  22: 3.3134380228069213,
  23: 3.464048842025418,
  24: 3.6146596612439144,
  25: 3.7652704804624104,
  26: 3.915881299680907,
  27: 4.066492118899403,
  28: 4.2171029381178995,
  29: 4.367713757336396,
  30: 4.5183245765548925,
  31: 4.6689353957733895,
  32: 4.8195462149918855,
  33: 4.970157034210382,
  34: 5.1207678534288785,
  35: 5.271378672647375,
  36: 5.4219894918658715,
  37: 5.572600311084368,
  38: 5.723211130302864,
  

In [8]:
len(milp_grid)


79

In [9]:
sq_evals = {'Carbon Offset': projections_2m[1].objective_projections['Carbon Offset'][2000000], 
            'Energy Potential': projections_2m[1].objective_projections['Energy Potential'][2000000], 
            'Racial Equity': projections_2m[1].objective_projections['Racial Equity'][2000000], 
            'Income Equity': projections_2m[1].objective_projections['Income Equity'][2000000],
            'label':"Status Quo"}
eva_result = {'Carbon Offset': 1.4290913608295026 * sq_evals['Carbon Offset'], 
            'Energy Potential': 1.1030419922110213 * sq_evals['Energy Potential'], 
            'Racial Equity': 1.3191408894477306 * sq_evals['Racial Equity'], 
            'Income Equity': 1.39675444243911 * sq_evals['Income Equity'],
            'label': "NEAT-Tournament (ours)",
            'color': "palegreen"}
create_pareto_front_plots(milp_grid, 'Carbon Offset', 'Energy Potential', others = [eva_result], scale=sq_evals, fit=2)


100%|██████████| 79/79 [00:00<00:00, 628.87it/s]


79


In [10]:
#code to edit a projection
# projections[1] = create_future_estimate_projection(zips_df, state_df, n_panels=num_panels, objectives=create_paper_objectives())
# #save
# with open(proj_path2, 'wb') as dir:
#     pickle.dump(projections, dir, pickle.HIGHEST_PROTOCOL)

In [11]:
compared_projections_2m = [projections_2m[1], projections_2m[2], projections_2m[3]] + [neat_proj_new_2m] #includes future installs, carbon-optimal, energy-optimal, and neat
# compared_projections_10m = [projections_10m[1], projections_10m[2], projections_10m[3]] + [neat_proj_new_10m] #includes future installs, carbon-optimal, energy-optimal, and neat

In [12]:
#color palettes

# methods_colorpalette = sns.color_palette("colorblind")[0:3] + sns.color_palette("colorblind")[4:] #skip the other orange color

# methods_colorpalette = sns.color_palette("colorblind")[0:1] + sns.color_palette("Set2")[0:2] + sns.color_palette("Set2")[3:4]

# methods_colorpalette = ["#0000a2","#bc272d", "#e9c716","#50ad9f"]

methods_colorpalette = ["#4a2377","#8cc5e3","#0d7d87", "#f55f74"]

# methods_colorpalette = ["#d31f11","#f47a00", "#62c8d3","#007191"]

# methods_colorpalette = ["#d31f11","#00a303ba", "#a14ca26a","#077A9A"]

In [13]:
#fig 3
plot_projections(compared_projections_2m, "Carbon Offset", color_palette=methods_colorpalette, interval=100000)
plot_projections(compared_projections_2m, "Energy Potential", color_palette=methods_colorpalette, interval=100000)

In [14]:
# plot_comparison_ratio(projections_10m[1], neat_proj_new_10m, "base","comp", interval=100000) 

In [ ]:
#fig 4
coverage_ratio = 674914/1600000 #ratio of panels covered by our dataset
annual_new_panels = [coverage_ratio * 720000 * 1.09**i for i in range(11)]

#get the baseline
installations2025 = 1973556
# projection2025 = create_future_estimate_projection(zips_df, state_df, installations2025, create_paper_objectives(), intervals=2)
baseline2025 = {key: projection2025.objective_projections[key][installations2025] for key in projection2025.objective_projections.keys()}
# print(baseline2025)

for num_panels in [sum(annual_new_panels[:i]) for i in [1,2,5]]:
    plot_bar_comparison_baseline_ratio(baseline2025, [projections_2m[2], projections_2m[3], neat_proj_new_2m], panel_count=num_panels, color_palette = methods_colorpalette, hatches=["\\","/","x"], hatch_size=0.5, fontsize=22)
# plot_bar_comparison_ratio(projections_2m[1], [projections_2m[2], projections_2m[3], neat_proj_new_2m], panel_count=1000000, color_palette = methods_colorpalette, hatches=["\\","/","x"], hatch_size=0.5, fontsize=22)
# plot_bar_comparison_ratio(projections_2m[1], [projections_2m[2], projections_2m[3], neat_proj_new_2m], panel_count=2000000, color_palette = methods_colorpalette, hatches=["\\","/","x"], hatch_size=0.5, fontsize=22)
# plot_bar_comparison_ratio(projections_10m[1], [projections_10m[2], projections_10m[3], neat_proj_new_10m], panel_count=10000000, color_palette = methods_colorpalette, hatches=["\\","/","x"], hatch_size=0.5)


#2 mil 
# plot_bar_comparison_ratio(projections_2m[1], [projections_2m[2], projections_2m[3], neat_proj_new_2m], panel_count=2000000, color_palette = methods_colorpalette, hatches=["\\","/","x"], hatch_size=0.5, fontsize=22)

303711.3
Carbon Optimized results: [np.float64(1.2651033942259082), np.float64(1.146291963659523), np.float64(0.9874380491945722), np.float64(1.0720339995682313)]
Energy Optimized results: [np.float64(1.1094289756776645), np.float64(1.1961463322355403), np.float64(1.0452263200923564), np.float64(1.108389619307412)]
EVA results: [np.float64(1.2568262473440188), np.float64(1.1689877502561319), np.float64(1.0702518690470524), np.float64(1.165040236880775)]
634756.6170000001
Carbon Optimized results: [np.float64(1.5486585704108347), np.float64(1.313942357055807), np.float64(1.032020222201648), np.float64(1.135734325621484)]
Energy Optimized results: [np.float64(1.2595664750365432), np.float64(1.4089031732245525), np.float64(1.0341966885279192), np.float64(1.1378422654648614)]
EVA results: [np.float64(1.5187369237458177), np.float64(1.3451180933151898), np.float64(1.116008331516186), np.float64(1.214908606733605)]
1817624.2394868934
Carbon Optimized results: [np.float64(2.528324391533766), 

In [35]:
#get absolute projected objectives from EVA after 5 years (2030) and 10 years (2035)
for i in [5,10]:
    num_panels = sum(annual_new_panels[:i])
    proj1 = projections_2m[1].interpolateObjectiveProjections(num_panels)
    proj2 = neat_proj_new_2m.interpolateObjectiveProjections(num_panels)
    print("Years:", i)
    print("SQ Carbon offset:",proj1['Carbon Offset'])
    print("EVA Carbon offset:",proj2['Carbon Offset'])
    print("Carbon ratio:", proj2['Carbon Offset']/proj1['Carbon Offset'])

    print("SQ Energy Potential:",proj1['Energy Potential'])
    print("EVA Energy Potential:",proj2['Energy Potential'])
    print("Energy ratio:", proj2['Energy Potential']/proj1['Energy Potential'])
    print("----")

Years: 5
SQ Carbon offset: 279797.76400810695
EVA Carbon offset: 405777.69967177266
Carbon ratio: 1.4502535469154625
SQ Energy Potential: 2049736072.1784184
EVA Energy Potential: 2243562674.232418
Energy ratio: 1.0945617363546736
----
Years: 10
SQ Carbon offset: 307871.9549724879
EVA Carbon offset: 443355.98776588624
Carbon ratio: 1.4400661723329282
SQ Energy Potential: 2255401339.450712
EVA Energy Potential: 2474296003.479157
Energy ratio: 1.097053531094273
----


In [16]:
# plot_equity_comparison(neat_proj_new, create_equity_objectives(), panel_counts = [0,100000,1000000], color_palette=sns.color_palette("Greens", 3)) #fig 4

In [17]:
# palette = sns.color_palette("Set1")
# sns.palplot(palette)

In [18]:
plot_bar_comparison_ratio(projections_2m[1], [projections_2m[2], neat_proj_new_2m, milp_proj], panel_count=num_panels, color_palette = methods_colorpalette, hatches=["\\","/","x"], hatch_size=0.5, fontsize=22)

NameError: name 'milp_proj' is not defined